In [1]:
import os 
import sys 
import json 
import logging 
from pathlib import Path 
from functools import cache
from itertools import product

# Required when developing in a jupyter-notebook environment 
cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from palettable.scientific.sequential import Batlow_5
from palettable.tableau import Tableau_10
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy

# Required when developing in a jupyter-notebook environment 
load_dotenv('../../../../.env')

print(os.environ['SUBGRAPH_URL'])
# logging.basicConfig(level=logging.DEBUG)

from utils_notebook.utils import ddf, remove_prefix, load_subgraph, remove_keys
from utils_notebook.vega import (
    output_chart, 
    apply_css, 
    wide_to_longwide, 
    chart, 
)
from utils_notebook.testing import validate_season_series
from utils_notebook.constants import ADDR_BEANSTALK
from utils_notebook.queries import QueryManager
from utils_notebook.css import css_tooltip_timeseries_multi_colored

https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk


In [3]:
ADDR_BEANSTALK.lower()

'0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5'

In [4]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs) 

In [28]:
farmers = bs.Query.farmers(
    first=100000, 
    where= {'silo_': {'id_not': "0xc1e088fc1323b20bcbee9bd1b9fc9546db5624c5"}}
)
df_farmers = sg.query_df(
    [
        farmers.id, 
        farmers.deposits.bdv, 
        farmers.deposits.removedBDV, 
    ], 
    pagination_strategy=ShallowStrategy
)
df_farmers = remove_prefix(df_farmers, "farmers_")
df_farmers = remove_prefix(df_farmers, "deposits_")

In [29]:
df_farmers.bdv = (df_farmers.bdv - df_farmers.removedBDV) / 1e6 
df_farmers = df_farmers.drop(columns=['removedBDV']) 
df_farmers = df_farmers.rename(columns={"id": "address"})
assert df_farmers.bdv.min() >= 0
df_farmers.head()

,address,bdv
0,0x0000000002ce79aacd54227d2163ff3791338975,0.000000
1,0x0000002e4f99cb1e699042699b91623b1334d2f7,0.000000
2,0x0000002e4f99cb1e699042699b91623b1334d2f7,0.000000
3,0x0000002e4f99cb1e699042699b91623b1334d2f7,0.000167
4,0x0000002e4f99cb1e699042699b91623b1334d2f7,0.000000


In [45]:
from functools import partial 

breakpoints = [
    1, 1e3, .5e4, 1e4, 2.5e4, 5e4, 1e5, 2.5e5, 5e5, 1e6, 5e6, float("inf") 
]

def classify(order, bdv):
    for i in range(1, len(breakpoints)):
        b0 = breakpoints[i-1] 
        b1 = breakpoints[i]
        if b0 <= bdv < b1: 
            if order: 
                return i 
            else: 
                if b1 == float('inf'): 
                    return f"{int(b0):,}+ bdv"
                else: 
                    return f"{int(b0):,} - {int(b1):,} bdv"
            
df_class_order = pd.DataFrame(data=[
    {'class': classify(False, breakpoints[i]), 'order': i}   
    for i in range(len(breakpoints))
]) 
df = df_farmers.groupby(by="address").sum().reset_index()
df['class'] = df.bdv.apply(partial(classify, False))
df['order'] = df.bdv.apply(partial(classify, True))
df = df.sort_values('bdv').reset_index(drop=True)
df = df.dropna(subset="class")
df.tail()

,address,bdv,class,order
2317,0xfb4570e63a5b87fd415ec6f855373902cae7a1f7,7.525624e+05,"500,000 - 1,000,000 bdv",9.0
2318,0xc06320d9028f851c6ce46e43f04aff0a426f446c,8.949305e+05,"500,000 - 1,000,000 bdv",9.0
2319,0xef764bac8a438e7e498c2e5fccf0f174c3e3f8db,9.276108e+05,"500,000 - 1,000,000 bdv",9.0
2320,0x10bf1dcb5ab7860bab1c3320163c6dddf8dcc0e4,1.432913e+06,"1,000,000 - 5,000,000 bdv",10.0
2321,0xc46c1b39e6c86115620f5297e98859529b92ad14,1.943953e+06,"1,000,000 - 5,000,000 bdv",10.0


In [46]:
# Get the count of pod holders by classification 
df_count_class = (
    df[['class', 'bdv']]
    .groupby('class').count()
    .reset_index()
    .merge(df_class_order, how="left", on="class")
    .rename(columns={"bdv": "count"})
)
# Sum value held by each class of holders 
df_class_value = (
    df[['class', 'bdv']]
    .groupby('class').sum()
    .reset_index()
    .merge(df_class_order, how="left", on="class")
)
color_domain = list(sorted(df_count_class['class'].unique()))
color_range = [Tableau_10.hex_colors[i] for i in range(len(color_domain))]
df_count_class.head()

,class,count,order
0,"1 - 1,000 bdv",934,0
1,"1,000 - 5,000 bdv",531,1
2,"1,000,000 - 5,000,000 bdv",2,9
3,"10,000 - 25,000 bdv",201,3
4,"100,000 - 250,000 bdv",53,6


In [48]:
width = 500 
x = alt.X("class:O", sort=alt.SortField("order"), axis=alt.Axis(title="Classification"))
color = alt.Color(
    "class:O", 
    legend=None, 
    scale=alt.Scale(domain=color_domain, range=color_range)
)
selection = alt.selection_single(
    encodings=['x'], nearest=True, on='mouseover', empty='none', clear='mouseout'
)

# Chart count class 
base_count_class = (
    alt.Chart(df_count_class, width=width, title="Count of Holders by BDV Held Classification")
    .mark_bar()
    .encode(x=x, y=alt.Y("count:Q", axis=alt.Axis(title="Unique Addresses")))
)
chart_count_class_histogram = (
    base_count_class
    .encode(
        color=color, 
        stroke=alt.condition(selection, alt.value("black"), alt.value("white"))
    )
    .mark_bar()
)
chart_count_class_text = (
    base_count_class
    .encode(
        text=alt.Text("count:Q", format=",d"),
        stroke=alt.value("black"), 
        strokeWidth=alt.condition(selection, alt.value(.6), alt.value(0)),
    )
    .mark_text(color='black', dy=-10)
)

# Chart class value 
base_class_value = (
    alt.Chart(df_class_value, width=width, title="Cumulative BDV by BDV Held Classification")
    .mark_bar()
    .encode(x=x, y=alt.Y("bdv:Q", axis=alt.Axis(title="BDV")),)
)
chart_class_value_histogram = (
    base_class_value
    .encode(
        color=color,
        stroke=alt.condition(selection, alt.value("black"), alt.value("white")),
    )
    .mark_bar()
)
chart_class_value_text = (
    base_class_value
    .encode(
        text=alt.Text("bdv:Q", format=".3s"),
        stroke=alt.value("black"), 
        strokeWidth=alt.condition(selection, alt.value(.6), alt.value(0)),
    )
    .mark_text(color='black', dy=-10)
)

c = (
    alt.hconcat(
        alt.layer(chart_count_class_histogram, chart_count_class_text).add_selection(selection), 
        alt.layer(chart_class_value_histogram, chart_class_value_text).add_selection(selection), 
    )
)
c

alt.HConcatChart(...)

In [49]:
output_chart(c)

<IPython.core.display.JSON object>